In [11]:
from env import config
import requests


In [12]:
# MINDSDB_UN=config("MINDSDB_UN", default=None) # oliver@mindsdb.com
# MINDSDB_PW=config("MINDSDB_PW", default=None)
# assert MINDSDB_UN is not None
# assert MINDSDB_PW is not None


MINDSDB_BASE_URL = 'http://127.0.0.1:47334/api/sql/query'


In [13]:
flightDate="2022-04-21"
sql_query=f"""
SELECT m.segmentsAirlineName, m.totalFare, m.totalFare_confidence FROM mindsdb.flight_prices_predictor AS m
JOIN ai_travel_agent.flight_prices AS t
WHERE t.flightDate > "{flightDate}"
AND t.startingAirport = "SFO"
AND t.isNonStop = 1
AND t.destinationAirport = "BOS";
"""

In [14]:
# # session = get_mindsdb_session()
# session = requests.session()

# def mindsdb_query(sql_query):
#      url = f"{MINDSDB_BASE_URL}";
#      r = session.post(url, json={"query": sql_query})
#      return r.json()

In [15]:
# session = get_mindsdb_session()
# query_response = mindsdb_query(sql_query)
# session.close()

In [16]:

session  = requests.Session()
def mindsdb_query(session, sql_query):
  url = f"{MINDSDB_BASE_URL}";

  return session.post(url, json={'query': sql_query})


In [17]:
def predict_query(session, flightDate="2022-04-21", startingAirport="SFO", isNonStop=1, destinationAirport="BOS", raw_request=True):
    sql_query=f"""
    SELECT m.flightDate,  m.segmentsAirlineName, m.isNonStop, m.totalFare, m.totalFare_confidence FROM mindsdb.flight_prices_predictor AS m
    JOIN ai_travel_agent.flight_prices AS t
    WHERE t.flightDate > "{flightDate}"
    AND t.startingAirport = "{startingAirport}"
    AND t.isNonStop = "{isNonStop}"
    AND t.destinationAirport = "{destinationAirport}";
    """
    response = mindsdb_query(session, sql_query)
    response.raise_for_status()
    if raw_request:
        return response
    data = response.json()
    columns = data.get("column_names")
    dataset = data.get('data')
    web_ready_data = [dict(zip(columns, row)) for row in dataset]
    return web_ready_data
    


In [18]:
query_response = predict_query(session,flightDate="2022-04-21", startingAirport="SFO", isNonStop=1, destinationAirport="BOS", raw_request=False)
session.close()

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=47334): Max retries exceeded with url: /api/sql/query (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x111ccf750>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [19]:
query_response

NameError: name 'query_response' is not defined

In [22]:
columns = data.get("column_names")
dataset = data.get('data')
columns

['segmentsAirlineName', 'isNonStop', 'totalFare', 'totalFare_confidence']

In [23]:
dataset

[['American Airlines', '1', 75.91602057000924, 0.9991],
 ['American Airlines', '1', 52.72806786553402, 0.9991],
 ['American Airlines', '1', 11.910481227118163, 0.9991],
 ['American Airlines', '1', 1.1904435869539636, 0.9991],
 ['American Airlines', '1', 4.4711619659959965, 0.9991],
 ['American Airlines', '1', 8.238788736663334, 0.9991],
 ['American Airlines', '1', 0.3012584562389968, 0.9991],
 ['American Airlines', '1', 24.174454901520416, 0.9991],
 ['American Airlines', '1', 32.78493103187807, 0.9991],
 ['American Airlines', '1', 44.683813399855175, 0.9991],
 ['Delta', '1', 40.78329980268202, 0.9991],
 ['Delta', '1', 64.30719314703386, 0.9991],
 ['Delta', '1', 17.180500829580872, 0.9991],
 ['Delta', '1', 8.385364065141832, 0.9991],
 ['Delta', '1', 13.017348043643535, 0.9991],
 ['Delta', '1', 2.065504252409335, 0.9991],
 ['Delta', '1', 10.445847287387789, 0.9991],
 ['Delta', '1', 36.39937818919746, 0.9991],
 ['Delta', '1', 29.7677818966123, 0.9991],
 ['Delta', '1', 42.33767510824158, 0

In [24]:
# web_ready_data = []
# for row in dataset:
#     print(row)
#     for i, col in enumerate(row):
#         columns[i], col

['American Airlines', '1', 75.91602057000924, 0.9991]
['American Airlines', '1', 52.72806786553402, 0.9991]
['American Airlines', '1', 11.910481227118163, 0.9991]
['American Airlines', '1', 1.1904435869539636, 0.9991]
['American Airlines', '1', 4.4711619659959965, 0.9991]
['American Airlines', '1', 8.238788736663334, 0.9991]
['American Airlines', '1', 0.3012584562389968, 0.9991]
['American Airlines', '1', 24.174454901520416, 0.9991]
['American Airlines', '1', 32.78493103187807, 0.9991]
['American Airlines', '1', 44.683813399855175, 0.9991]
['Delta', '1', 40.78329980268202, 0.9991]
['Delta', '1', 64.30719314703386, 0.9991]
['Delta', '1', 17.180500829580872, 0.9991]
['Delta', '1', 8.385364065141832, 0.9991]
['Delta', '1', 13.017348043643535, 0.9991]
['Delta', '1', 2.065504252409335, 0.9991]
['Delta', '1', 10.445847287387789, 0.9991]
['Delta', '1', 36.39937818919746, 0.9991]
['Delta', '1', 29.7677818966123, 0.9991]
['Delta', '1', 42.33767510824158, 0.9991]
['Alaska Airlines', '1', 66.8445

In [25]:
web_ready_data = [dict(zip(columns, row)) for row in dataset]
web_ready_data


[{'segmentsAirlineName': 'American Airlines',
  'isNonStop': '1',
  'totalFare': 75.91602057000924,
  'totalFare_confidence': 0.9991},
 {'segmentsAirlineName': 'American Airlines',
  'isNonStop': '1',
  'totalFare': 52.72806786553402,
  'totalFare_confidence': 0.9991},
 {'segmentsAirlineName': 'American Airlines',
  'isNonStop': '1',
  'totalFare': 11.910481227118163,
  'totalFare_confidence': 0.9991},
 {'segmentsAirlineName': 'American Airlines',
  'isNonStop': '1',
  'totalFare': 1.1904435869539636,
  'totalFare_confidence': 0.9991},
 {'segmentsAirlineName': 'American Airlines',
  'isNonStop': '1',
  'totalFare': 4.4711619659959965,
  'totalFare_confidence': 0.9991},
 {'segmentsAirlineName': 'American Airlines',
  'isNonStop': '1',
  'totalFare': 8.238788736663334,
  'totalFare_confidence': 0.9991},
 {'segmentsAirlineName': 'American Airlines',
  'isNonStop': '1',
  'totalFare': 0.3012584562389968,
  'totalFare_confidence': 0.9991},
 {'segmentsAirlineName': 'American Airlines',
  'is